<a href="https://colab.research.google.com/github/godmin18/NLP_Portfolio/blob/master/NLI_bert_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://heung-bae-lee.github.io/2020/01/21/deep_learning_10/

http://yonghee.io/bert_binary_classification_naver/

https://github.com/deepseasw/bert-naver-movie-review/blob/master/bert_naver_movie.ipynb

https://sanghyu.tistory.com/113

In [1]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 27.0 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 6.8 MB 38.1 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import os, sys 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import tensorflow as tf

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

#Basic
import pandas as pd
import numpy as np
import random
import time
import datetime
import matplotlib.pyplot as plt

In [4]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
  print(torch.cuda.get_device_name(i))

1
Tesla K80


In [5]:
DIR = '/content/drive/MyDrive/project/자연어추론(NLI)/한국어문장관계분류/data'
TRAIN_SOURCE = os.path.join(DIR, "train_data.csv")
TEST_SOURCE = os.path.join(DIR, "test_data.csv")
SAMPLE_SUBMISSION = os.path.join(DIR, "sample_submission.csv")

train = pd.read_csv(TRAIN_SOURCE)
test = pd.read_csv(TEST_SOURCE)
print("train row : {}, train col : {}".format(str(train.shape[0]), str(train.shape[1])))
print("test row : {}, test col : {}".format(str(test.shape[0]), str(test.shape[1])))

train row : 24998, train col : 4
test row : 1666, test col : 4


In [ ]:
train.head(10)

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
24993,24993,"오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 이스토리쿠...",오라토리오에서 테스토의 역할이 가장 중요하다.,neutral
24994,24994,지하철역까지 걸어서 5분 정도 걸립니다.,지하철역까지 도보로 5분 정도 걸립니다.,entailment
24995,24995,한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과를 공개했다.,중악방역대책본부는 집단 감염과 관련한 모든 정보를 비공개했다.,contradiction
24996,24996,마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.,시청자들은 마미손의 정체를 안다.,entailment


In [20]:
labels = train['label'].apply(lambda x : 0 if x =='contradiction' else 1 if x =='entailment' else 2).values

In [14]:
# tokenizer
tokenizer = BertTokenizer.from_pretrained("klue/bert-base", do_lower_case=False) #최종 8.1
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

In [7]:
MAX_LEN = 210

In [6]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids = [], [], []

    for sent1, sent2 in zip(sent_list1, sent_list2):
        encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, pad_to_max_length=True, )

        input_ids.append(encoding_result['input_ids'])
        attention_masks.append(encoding_result['attention_mask'])
        token_type_ids.append(encoding_result['token_type_ids'])

    return (input_ids, attention_masks, token_type_ids)

In [15]:
X_train = convert_examples_to_features(train['premise'], train['hypothesis'], max_seq_len=MAX_LEN,  tokenizer=tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [65]:
# decode 확인
def decode_input(input_ids):
  if type(input_ids) == list:
    decoded = tokenizer.decode(input_ids)
    print(decoded)
  else:
    wrong_type = type(input_ids)
    raise TypeError("'input_ids' should be list, type is {}".format(wrong_type))

In [16]:
# input_values for bert model
input_ids = X_train[0]
attention_masks = X_train[1]
token_type_ids = X_train[2]

In [ ]:
# reference: https://albertauyeung.github.io/2020/06/19/bert-tokenization.html/
sample_sentence = input_ids[0]
print(sample_sentence)
print(tokenizer.convert_ids_to_tokens(sample_sentence)) #wordpiece embedding

In [ ]:
# 패딩을 제외한 토큰들은 1로 표현
print(attention_masks[0])

In [ ]:
# 입력된 두문장을 1과 0으로 구분하기 위함.
print(token_type_ids[0])

In [21]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, train_tokens, validation_tokens = train_test_split(attention_masks, 
                                                       token_type_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

In [22]:
# 리스트를 입력 받아 파이토치의 텐서로 변환 (메모리 공간을 효율적으로 사용하기 위해)
# https://hiddenbeginner.github.io/deeplearning/2020/01/21/pytorch_tensor.html
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_tokens = torch.tensor(train_tokens)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_tokens = torch.tensor(validation_tokens)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(train_tokens[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])
print(validation_tokens[0])

tensor([    2,  4132,  9417,  2138,  4466, 12551,  2119,  5316,  2116,  1415,
         2359,  5515,    18,     3,  4132,  9417,  2299,  2118,  4672,  4352,
         2052,  1521,   859,  2689,  2359,  5515,    18,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
# 배치 사이즈
batch_size = 32

# train_inputs, train_masks, train_tokens, train_labels 각각의 텐서를 튜플로 묶음
train_data = TensorDataset(train_inputs, train_masks, train_tokens, train_labels)
train_sampler = RandomSampler(train_data)

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_tokens, validation_labels,)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [8]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [9]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [10]:
model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=3)
# model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model.cuda()

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [23]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  # lr = 5e-6,
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


NameError: ignored

In [24]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 연습
model.train()
input = train_inputs[0].unsqueeze(0).to(device)
mask = train_masks[0].unsqueeze(0).to(device)
token = train_tokens[0].unsqueeze(0).to(device)
label = train_labels[0].unsqueeze(0).to(device)

outputs = model(input, token_type_ids=token, attention_mask=mask, labels = label)

In [ ]:
# model 출력 살펴보기
outputs

In [ ]:
# outputs은 loss와 logits의 두 key 값을 반환
outputs.keys()

In [ ]:
# logits의 결과는 3개의 값으로 표현
logits = outputs[1][0]
print(logits)

# argmax는 가장 큰 값의 index를 반환
logits = logits.detach().cpu().numpy()
print(np.argmax(logits, axis=0))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

train_losses = []
valid_losses = []

for epoch_i in range(0, epochs):
  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

  t0 = time.time()

  # 로스 초기화
  train_loss = 0

  # 모델 훈련모드 돌입 (규제 사용)
  model.train()

  for step, batch in enumerate(train_dataloader):
    if step % 32 == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
      print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

    #batch를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 그래디언트 초기화
    model.zero_grad()

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_token, b_labels = batch

    # Forward 수행                
    outputs = model(b_input_ids, 
                    token_type_ids=b_token, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
    # 로스 구함
    loss = outputs[0]

    # 총 로스 계산
    train_loss += loss.item()
    

    # Backward 수행으로 그래디언트 계산 (loss를 기반으로 적절한 파라미터를 계산)
    loss.backward()

    # 그래디언트를 통해 가중치 파라미터 업데이트
    optimizer.step()
    
    # 스케줄러로 학습률 감소
  scheduler.step()
  
  # 평균 로스 계산
  avg_train_loss = train_loss / len(train_dataloader)

  train_losses.append(train_loss)

  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))
  print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))


  # ========================================
  #               Validation
  # ========================================

  print("")
  print("Running Validation...")

  #시작 시간 설정
  t0 = time.time()

  # 평가모드로 변경
  model.eval()

  # 변수 초기화
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask, b_token, b_labels  = batch

    # 그래디언트 계산 안함
    with torch.no_grad():     
      # Forward 수행
      outputs = model(b_input_ids, 
                      token_type_ids=b_token, 
                      attention_mask=b_input_mask, 
                      labels=b_labels)
      
    # 로스 구함
    loss = outputs[0]
    eval_loss += loss.item()
    valid_losses.append(eval_loss)

    # 정확도 구함
    logits = outputs[1]
    

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
      
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
        
  print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
  print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

In [ ]:
plt.plot(train_losses) 
# plt.plot(valid_losses)

In [ ]:
# torch.save(model, '/content/drive/MyDrive/Colab Notebooks/한국어문장관계분류/data/model.pt')

In [12]:
model = torch.load('/content/drive/MyDrive/project/자연어추론(NLI)/한국어문장관계분류/etc/model.pt')

In [61]:
def convert_input_data(sent_list1, sent_list2, max_seq_len):

    encoding_result = tokenizer.encode_plus(sent_list1, sent_list2, max_length=max_seq_len, pad_to_max_length=True, )

    input_ids = encoding_result['input_ids']
    attention_masks = encoding_result['attention_mask']
    token_type_ids = encoding_result['token_type_ids']

    return (input_ids, attention_masks, token_type_ids)

In [117]:
def bert_sentence(sent1, sent2):
  model.eval()
  input, mask, token = convert_input_data(sent1, sent2, 210)

  input = torch.tensor(input)
  mask = torch.tensor(mask)
  token = torch.tensor(token)

  test_input_ = input.unsqueeze(0).to(device)
  test_masks_ = mask.unsqueeze(0).to(device)
  test_tokens_ = token.unsqueeze(0).to(device)

  with torch.no_grad(): 
      # Forward 수행
      outputs = model(test_input_, 
                      token_type_ids=test_tokens_, 
                      attention_mask=test_masks_)
      
  logits = outputs[0]

  # CPU로 데이터 이동
  logits = logits.detach().cpu().numpy()

  result = np.argmax(logits)

  return result

In [118]:
test_list = []
for sentence_pair in  test[['premise', 'hypothesis']].values:
  sent1 = sentence_pair[0]
  sent2 = sentence_pair[1]

  test_list.append(bert_sentence(sent1, sent2))


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [119]:
len(test_list)

1666

In [120]:
sample_submission = pd.read_csv(SAMPLE_SUBMISSION)

In [121]:
sample_submission['label'] = test_list
sample_submission['label'].replace({0: 'contradiction', 1: 'entailment', 2: 'neutral'}, inplace=True)
sample_submission.to_csv('/content/drive/MyDrive/project/자연어추론(NLI)/한국어문장관계분류/etc/sample_submission_Bert2.csv', index=False)